## 準備一些實驗資料

In [ ]:
boxes_0 = []
with open("boxes0.txt", 'r', encoding='utf-8') as f:
  datas = f.readlines()

for data in datas:
  box = data.split(',')
  boxes_0.append([int(box[0]), int(box[1]), int(box[2]), int(box[3])])

print(boxes_0)

[[971, 310, 54, 120], [500, 296, 47, 116], [944, 475, 69, 116], [757, 629, 70, 91], [443, 431, 54, 141], [761, 333, 42, 104]]


In [ ]:
boxes_1 = []
with open("boxes1.txt", 'r', encoding='utf-8') as f:
  datas = f.readlines()

for data in datas:
  box = data.split(',')
  boxes_1.append([int(box[0]), int(box[1]), int(box[2]), int(box[3])])

print(boxes_1)

In [ ]:
# 讀入frame 2, frame 3的bounding boxes
boxes_2 = []
with open("boxes2.txt", 'r', encoding='utf-8') as f:
  datas = f.readlines()
for data in datas:
  box = data.split(',')
  boxes_2.append([int(box[0]), int(box[1]), int(box[2]), int(box[3])])
boxes_3 = []
with open("boxes3.txt", 'r', encoding='utf-8') as f:
  datas = f.readlines()
for data in datas:
  box = data.split(',')
  boxes_3.append([int(box[0]), int(box[1]), int(box[2]), int(box[3])])

print(boxes_2)
print(boxes_3)

---

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 設定一個合理的移動距離
DISTANCE_TH = 13.0

# 設定信心度下降率
CONFIDENCE_DROP_RATE = 0.9

# 設定信心度TH
CONFIDENCE_THRESHOLD = 0.5

---
# Utility Functions

In [ ]:
# 取得bounding box中心點
def get_center(box):
    cx = box[0] + box[2] // 2
    cy = box[1] + box[3] // 2
    return [cx, cy]

In [ ]:
# 取得兩點間歐式距離
def get_distance(position_1, position_2):
    vector1 = np.array(position_1)
    vector2 = np.array(position_2)
    dist = np.linalg.norm(vector1-vector2)
    return dist

In [ ]:
# Tracking table initialization with ID
def tracking_table_init_with_id(boxes):
    '''
        只會用在第一個frame

        build TABLE as
        [ {ID::int, POSITION::[x, y], CONFIDENCE::float} ]
    '''
    tracking_table = []
    for i, box in enumerate(boxes):
        item = {'id': i, 'pos': get_center(box), 'confidence': 1.0}
        tracking_table.append(item)
    return tracking_table

# Tracking table initialization without ID
def tracking_table_init(boxes):
    '''
        只填入中心點座標

        build TABLE as
        [ {ID::int, POSITION::[x, y], CONFIDENCE::float} ]
    '''
    tracking_table = []
    for i, box in enumerate(boxes):
        item = {'id': None, 'pos': get_center(box), 'confidence': None}
        tracking_table.append(item)
    return tracking_table

In [1]:
# 建立兩個frame每個box之間的距離表
# 輸入為:: TABLE
def build_dist_table_boxes(current_table, last_table):
    dist_table = []
    dist = []

    for box_n in current_table:
        for box_o in last_table:  # get one row
            dist.append(round(get_distance(box_o['pos'], box_n['pos']), 2))
        dist_table.append(dist)
        dist = []

    new_names = ['curr_{}' .format(i) for i in range(len(current_table))]
    old_names = ['last_{}' .format(i) for i in range(len(last_table))]
    df = pd.DataFrame(dist_table, index=new_names, columns=old_names)
    print("Dimension of DF=", df.shape)
    return df

---

# Restart from scratch

## STEP 0 - Initializing tables

#### 從Frame 0 開始的作法
> 需要加上FLAG控制

In [ ]:
# input data: boxes_2, boxes_3
old = tracking_table_init_with_id(boxes_2)

print(old)
print(new)
print("Length of old:", len(old))
print("Length of new:", len(new))

[{'id': 0, 'pos': [474, 487], 'confidence': 1.0}, {'id': 1, 'pos': [792, 671], 'confidence': 1.0}, {'id': 2, 'pos': [527, 346], 'confidence': 1.0}, {'id': 3, 'pos': [973, 533], 'confidence': 1.0}, {'id': 4, 'pos': [782, 379], 'confidence': 1.0}, {'id': 5, 'pos': [1000, 374], 'confidence': 1.0}]
[{'id': None, 'pos': [473, 480], 'confidence': None}, {'id': None, 'pos': [971, 534], 'confidence': None}, {'id': None, 'pos': [527, 342], 'confidence': None}, {'id': None, 'pos': [792, 669], 'confidence': None}, {'id': None, 'pos': [782, 373], 'confidence': None}]
Length of old: 6
Length of new: 5


#### 從Frame >=1 開始的作法
* old = new
* new = init(新的Frame)

In [ ]:
# old = new
# new = tracking_table_init(boxes_new)

## STEP 1 - Pairing
* 1-1 Generate dist_df
* 1-2 Loop for each column(old)
> * find min
> * pairing
> * Remove Column
> * Remove Row(if success)
* 1-3 Row remain in dist_table: give new ID & C=1

In [2]:
def do_pairing(new, old):
    '''
    * 1-1 Generate dist_df
    * 1-2 Loop for each column(old items)
        * find min
        * pairing
        * Remove Column
        * Remove Row(if success)
    * 1-3 Row remain in dist_table: give new ID & C=1
    '''
    # 1-1
    dist_df = build_dist_table_boxes(new, old)
    col_names = list(dist_df.columns)
    # row_names = list(dist_df.index)

    # 1-2
    for col_name in col_names:
        # Finding Min && 在THRESHOLD內
        min = dist_df[col_name].min()
        try:
            idx = dist_df[col_name].idxmin()
        except ValueError:  # len(old) > len(new)
            print("len(old) > len(new) ... End at {}" .format(col_name))
            # 將剩餘所有的old item加入new(作法與配對失敗相同)
            # (1) 取得剩餘的
            rest_of_cols = list(dist_df.columns)
            for col in rest_of_cols:
                idx_from_last = int(col.split('_')[-1])
                old[idx_from_last]['confidence'] *= CONFIDENCE_DROP_RATE
                new.append(old[idx_from_last])
            break

        # Pairing
        if min < DISTANCE_TH:
            idx_from_last = int(col_name.split('_')[-1])
            idx_target_current = int(idx.split('_')[-1])
            # 配對成功的處理
            # 更新 new table: (1)繼承ID (2)Confidence = 1
            new[idx_target_current]['id'] = old[idx_from_last]['id']
            new[idx_target_current]['confidence'] = 1
            print("(Pairing) new[{}] <-- old[{}] ID = {} ...... Distance = {}"
                  .format(idx_target_current, idx_from_last, old[idx_from_last]['id'], min))
        else:  # 沒配對成功 --> 處理舊的物件
            # 1. Ｃ * CONFIDENCE_DROP_RATE
            idx_from_last = int(col_name.split('_')[-1])
            old[idx_from_last]['confidence'] *= CONFIDENCE_DROP_RATE
            # 2. 加入(繼承)至current_table ...... 使用append實作
            new.append(old[idx_from_last])
            print("(pairing fail) new[{}] <--- old[{}] with C = {}"
                  .format(idx_from_last, idx_from_last, old[idx_from_last]['confidence']*CONFIDENCE_DROP_RATE))

        # 移除 Column
        dist_df = dist_df.drop(col_name, axis=1)
        # 移除 Row
        dist_df = dist_df.drop(idx)

    # 1-3 將current frame中未配對到的box指派新ID
    for i, item in enumerate(new):
        if item['id'] is None or item['confidence'] is None:
            # print("(None type ID) new[{}]: {}" .format(i, item))
            # ids: list出目前的所有id
            ids = [new[i]['id'] for i in range(len(new))]

            # 找一個在範圍內未使用的id來使用
            for j in range(len(new)):
                if j not in ids:
                    new[i]['id'] = j
                    new[i]['confidence'] = 1
                    break
            print("(new id) new[{}] = {}" .format(i, new[i]['id']))

## STEP 2 - Remove [ C < 0.5 ]

In [ ]:
def remove_low_confidence(table):
    # 找出要刪除的index
    l_to_remove = []
    for i in range(len(table)):
        if table[i]['confidence'] < CONFIDENCE_THRESHOLD:
            l_to_remove.append(i)

    # 要從尾往前刪
    for i in l_to_remove[::-1]:
        del table[i]

## STEP 3 - Table checking

In [ ]:
def none_type_checking(table):
    for i, item in enumerate(table):
        if item['id']==None or item['confidence']==None:
            print("Got None type field in item\nnew[{}]: {}" .format(i, item))

---
# Simulation
### From frame_0

In [3]:
frame0_flag = 0  # 代表目前為最初始狀態，從frame_0開

In [ ]:
if flag == 0:
    # INITIALIZATION
    new = tracking_table_init_with_id(boxes_0)
    frame0_flag = 1
else:
    # INITIALIZATION
    old = new
    new = tracking_table_init(boxes_3)  # 模擬改boxes_1, 2, 3, ...

    # TRACKING
    do_pairing(new, old)  # pairing
    remove_low_confidence(new)  # removing
    none_type_checking(new)  # checking

print(new)
df = pd.DataFrame(new)
df

last_0 與 curr_0 配對成功 - 距離：7.07
要被繼承的old table的index為: 0
要被更新的new table的index為: 0
last_0 與 curr_3 配對成功 - 距離：2.0
要被繼承的old table的index為: 1
要被更新的new table的index為: 3
last_0 與 curr_2 配對成功 - 距離：4.0
要被繼承的old table的index為: 2
要被更新的new table的index為: 2
last_0 與 curr_1 配對成功 - 距離：2.24
要被繼承的old table的index為: 3
要被更新的new table的index為: 1
last_0 與 curr_4 配對成功 - 距離：6.0
要被繼承的old table的index為: 4
要被更新的new table的index為: 4
比對結束於：last_5
[{'id': 4, 'pos': [473, 480], 'confidence': 1}, {'id': 2, 'pos': [971, 534], 'confidence': 1}, {'id': 1, 'pos': [527, 342], 'confidence': 1}, {'id': 3, 'pos': [792, 669], 'confidence': 1}, {'id': 5, 'pos': [782, 373], 'confidence': 1}, {'id': 0, 'pos': [1000, 374], 'confidence': 0.9}]


,id,pos,confidence
0,4,"[473, 480]",1.0
1,2,"[971, 534]",1.0
2,1,"[527, 342]",1.0
3,3,"[792, 669]",1.0
4,5,"[782, 373]",1.0
5,0,"[1000, 374]",0.9


In [ ]:
print(old)
df = pd.DataFrame(old)
df

[{'id': 4, 'pos': [474, 487], 'confidence': 1}, {'id': 3, 'pos': [792, 671], 'confidence': 1}, {'id': 1, 'pos': [527, 346], 'confidence': 1}, {'id': 2, 'pos': [973, 533], 'confidence': 1}, {'id': 5, 'pos': [782, 379], 'confidence': 1}, {'id': 0, 'pos': [1000, 374], 'confidence': 0.9}]


,id,pos,confidence
0,4,"[474, 487]",1.0
1,3,"[792, 671]",1.0
2,1,"[527, 346]",1.0
3,2,"[973, 533]",1.0
4,5,"[782, 379]",1.0
5,0,"[1000, 374]",0.9
